# Part IV: Data Cleaning

In [162]:
import os
import json
import requests
import pandas as pd
import openpyxl

# Read the data
us_data = pd.read_excel('data/US_EC1B1_Dataset.xlsx')
germany_data = pd.read_excel('data/Germany_EC1B1_Dataset.xlsx')

def reindex(df):
    # Set the first row (index 0) as the new header
    df.columns = df.iloc[0]  # Use the first row as the header
    df = df.drop(0)  # Drop the first row (since it was the header now)
    
    # Reset index after dropping the row
    df.reset_index(drop=True, inplace=True)
    
    # Name the first 2 columns
    df.columns = [f'Country' if i == 0 
                  else f'Month-Year' if i == 1
                  else col for i, col in enumerate(df.columns)]

    return df

us_df = reindex(us_data)
germany_df = reindex(germany_data)


## Merging the Datasets (Append)

In [163]:
# Get the union of the columns from both datasets
all_columns = list(set(germany_df.columns) | set(us_df.columns))

# Ensure both datasets have all columns, adding None for missing columns
germany_data = germany_df.reindex(columns=all_columns, fill_value=None)
us_data = us_df.reindex(columns=all_columns, fill_value=None)

# Append the datasets (one above the other)
combined_data = pd.concat([germany_data, us_data], ignore_index=True)
combined_data = combined_data[['Month-Year', 
                         'Country',
                         'International Reserves and Liquidity, Reserves, Official Reserve Assets, US Dollar',
                         'Prices, Consumer Price Index, All items, Index',
                         'Economic Activity, Industrial Production, Index',
                         'Exchange Rates, National Currency Per U.S. Dollar, Period Average, Rate']]

# Check the result
combined_data

,Month-Year,Country,"International Reserves and Liquidity, Reserves, Official Reserve Assets, US Dollar","Prices, Consumer Price Index, All items, Index","Economic Activity, Industrial Production, Index","Exchange Rates, National Currency Per U.S. Dollar, Period Average, Rate"
0,Dec 1959,Germany,4811.474341,24.616929,32.500305,4.2
1,Jan 1960,Germany,4724.155785,24.616929,31.193881,4.2
2,Feb 1960,Germany,4806.36283,24.477068,31.041599,4.2
3,Mar 1960,Germany,4966.456016,24.477068,32.203755,4.2
4,Apr 1960,Germany,5236.120624,24.616929,34.287622,4.2
...,...,...,...,...,...,...
741,Aug 1990,United States,78908.838357,60.351608,NaN,NaN
742,Sep 1990,United States,80024.166133,60.856066,NaN,NaN
743,Oct 1990,United States,82852.196532,61.222946,NaN,NaN
744,Nov 1990,United States,83059.402774,61.360525,NaN,NaN


## Data Formatting

In [164]:
# DATATYPE CONVERSION
# Convert the first column to datetime type
combined_data['Month-Year'] = pd.to_datetime(combined_data['Month-Year'], format='%b %Y')

# Convert the second column to string type
combined_data['Country'] = combined_data['Country'].astype(str)

# Convert the remaining columns to numeric type
combined_data.iloc[:, 2:] = combined_data.iloc[:, 2:].apply(pd.to_numeric, errors='coerce')

# print(final_data.dtypes)

# Filter monthly data for Germany
monthly_germany = combined_data[combined_data['Country'] == 'Germany']
monthly_us = combined_data[combined_data['Country'] == 'United States']

# Checking for NaN values throughout the data
for column in monthly_germany.columns:
    filtered_df = monthly_germany[monthly_germany[column].isnull()]

# print(filtered_df)

monthly_germany

,Month-Year,Country,"International Reserves and Liquidity, Reserves, Official Reserve Assets, US Dollar","Prices, Consumer Price Index, All items, Index","Economic Activity, Industrial Production, Index","Exchange Rates, National Currency Per U.S. Dollar, Period Average, Rate"
0,1959-12-01,Germany,4811.474341,24.616929,32.500305,4.2
1,1960-01-01,Germany,4724.155785,24.616929,31.193881,4.2
2,1960-02-01,Germany,4806.36283,24.477068,31.041599,4.2
3,1960-03-01,Germany,4966.456016,24.477068,32.203755,4.2
4,1960-04-01,Germany,5236.120624,24.616929,34.287622,4.2
...,...,...,...,...,...,...
368,1990-08-01,Germany,72425.738573,67.556702,75.964955,1.5707
369,1990-09-01,Germany,73197.573621,67.766509,86.496497,1.5697
370,1990-10-01,Germany,75011.92683,68.256049,92.643903,1.5233
371,1990-11-01,Germany,76166.074709,68.116177,89.461999,1.487


## Defining Variables (Germany)
### 1. Monthly Nominal Exchange Rate (NER)

In [165]:
# Monthly growth in nominal exchange rate (NER)

monthly_germany.loc[:, 'Monthly_Nominal_ER_Growth (%)'] = monthly_germany['Exchange Rates, National Currency Per U.S. Dollar, Period Average, Rate'].pct_change() * 100

monthly_germany[['Month-Year', 'Monthly_Nominal_ER_Growth (%)']]

C:\Users\srini\AppData\Local\Temp\ipykernel_11476\495105941.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monthly_germany.loc[:, 'Monthly_Nominal_ER_Growth (%)'] = monthly_germany['Exchange Rates, National Currency Per U.S. Dollar, Period Average, Rate'].pct_change() * 100


,Month-Year,Monthly_Nominal_ER_Growth (%)
0,1959-12-01,NaN
1,1960-01-01,0.0
2,1960-02-01,0.0
3,1960-03-01,0.0
4,1960-04-01,0.0
...,...,...
368,1990-08-01,-4.219769
369,1990-09-01,-0.063666
370,1990-10-01,-2.955979
371,1990-11-01,-2.382984


### 2. Monthly Real Exchange Rate (RER)

In [166]:
# CPI Ratio for every Month-Year
# Pivot table to create separate CPI columns for each country
df_pivot = combined_data.pivot(index='Month-Year', columns='Country', values='Prices, Consumer Price Index, All items, Index').reset_index()

# Rename columns for clarity
df_pivot.columns.name = None  # Remove index name
df_pivot.rename(columns={'Germany': 'CPI_Foreign', 'United States': 'CPI_Domestic'}, inplace=True)

# Calculate CPI ratio (Foreign / Domestic)
df_pivot['CPI_Ratio'] = df_pivot['CPI_Foreign'] / df_pivot['CPI_Domestic']

# Monthly growth in real exchange rate (RER)
# Ensure both DataFrames are aligned on 'Month-Year'

# Multiply the two columns
monthly_germany.loc[:, 'Monthly_Real_ER_Growth (%)'] = monthly_germany['Monthly_Nominal_ER_Growth (%)'] * df_pivot['CPI_Ratio']

monthly_germany[['Month-Year', 'Monthly_Real_ER_Growth (%)']]

C:\Users\srini\AppData\Local\Temp\ipykernel_11476\1829369591.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monthly_germany.loc[:, 'Monthly_Real_ER_Growth (%)'] = monthly_germany['Monthly_Nominal_ER_Growth (%)'] * df_pivot['CPI_Ratio']


,Month-Year,Monthly_Real_ER_Growth (%)
0,1959-12-01,NaN
1,1960-01-01,0.0
2,1960-02-01,0.0
3,1960-03-01,0.0
4,1960-04-01,0.0
...,...,...
368,1990-08-01,-4.723548
369,1990-09-01,-0.070895
370,1990-10-01,-3.295553
371,1990-11-01,-2.645345


### 3. Index of Real Exchange Rate (RER)

In [167]:
# Monthly Real ER
monthly_germany.loc[:, 'Monthly_Real_ER'] = monthly_germany['Prices, Consumer Price Index, All items, Index'] * df_pivot['CPI_Ratio']

# Index Real ER (assuming Real ER for 12-1990 is 1)

monthly_germany.loc[:, 'Real_ER_Index'] = monthly_germany['Monthly_Real_ER'] / monthly_germany['Monthly_Real_ER'].iloc[-1]

monthly_germany[['Month-Year', 'Monthly_Real_ER', 'Real_ER_Index']]


C:\Users\srini\AppData\Local\Temp\ipykernel_11476\1133379292.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monthly_germany.loc[:, 'Monthly_Real_ER'] = monthly_germany['Prices, Consumer Price Index, All items, Index'] * df_pivot['CPI_Ratio']
C:\Users\srini\AppData\Local\Temp\ipykernel_11476\1133379292.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monthly_germany.loc[:, 'Real_ER_Index'] = monthly_germany['Monthly_Real_ER'] / monthly_germany['Monthly_Real_ER'].iloc[-1]


,Month-Year,Monthly_Real_ER,Real_ER_Index
0,1959-12-01,44.945628,0.593177
1,1960-01-01,45.099026,0.595202
2,1960-02-01,44.436364,0.586456
3,1960-03-01,44.436364,0.586456
4,1960-04-01,44.79327,0.591167
...,...,...,...
368,1990-08-01,75.621979,0.998034
369,1990-09-01,75.46166,0.995918
370,1990-10-01,76.097093,1.004304
371,1990-11-01,75.61561,0.99795


### 4. Monthly Inflation Rate

In [168]:
# Monthly inflation rate

monthly_germany.loc[:, 'Monthly_Inflation_Rate (%)'] = monthly_germany['Prices, Consumer Price Index, All items, Index'].pct_change() * 100

monthly_germany[['Month-Year', 'Monthly_Inflation_Rate (%)']]

C:\Users\srini\AppData\Local\Temp\ipykernel_11476\2817474660.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monthly_germany.loc[:, 'Monthly_Inflation_Rate (%)'] = monthly_germany['Prices, Consumer Price Index, All items, Index'].pct_change() * 100


,Month-Year,Monthly_Inflation_Rate (%)
0,1959-12-01,NaN
1,1960-01-01,0.0
2,1960-02-01,-0.568147
3,1960-03-01,0.0
4,1960-04-01,0.571393
...,...,...
368,1990-08-01,0.311532
369,1990-09-01,0.310565
370,1990-10-01,0.722391
371,1990-11-01,-0.204922


### 5. Monthly Growth in Industrial Production

In [169]:
# Monthly Growth in Industrial Production

monthly_germany.loc[:, 'Monthly_Industrial_Growth (%)'] = monthly_germany['Economic Activity, Industrial Production, Index'].pct_change() * 100

monthly_germany[['Month-Year', 'Monthly_Industrial_Growth (%)']]

C:\Users\srini\AppData\Local\Temp\ipykernel_11476\4241067364.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monthly_germany.loc[:, 'Monthly_Industrial_Growth (%)'] = monthly_germany['Economic Activity, Industrial Production, Index'].pct_change() * 100


,Month-Year,Monthly_Industrial_Growth (%)
0,1959-12-01,NaN
1,1960-01-01,-4.019729
2,1960-02-01,-0.488181
3,1960-03-01,3.743868
4,1960-04-01,6.470881
...,...,...
368,1990-08-01,-3.590683
369,1990-09-01,13.863684
370,1990-10-01,7.107116
371,1990-11-01,-3.434553


### 6. Year on Year (YOY) Growth in Industrial Production

In [170]:
# Year on Year Growth in Industrial Production

monthly_germany.loc[:, 'YOY_Monthly_Industrial_Growth (%)'] = monthly_germany['Economic Activity, Industrial Production, Index'].pct_change(periods=12) * 100

monthly_germany[['Month-Year', 'YOY_Monthly_Industrial_Growth (%)']]

C:\Users\srini\AppData\Local\Temp\ipykernel_11476\3790083177.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monthly_germany.loc[:, 'YOY_Monthly_Industrial_Growth (%)'] = monthly_germany['Economic Activity, Industrial Production, Index'].pct_change(periods=12) * 100


,Month-Year,YOY_Monthly_Industrial_Growth (%)
0,1959-12-01,NaN
1,1960-01-01,NaN
2,1960-02-01,NaN
3,1960-03-01,NaN
4,1960-04-01,NaN
...,...,...
368,1990-08-01,6.017897
369,1990-09-01,5.514275
370,1990-10-01,6.152998
371,1990-11-01,5.580779


### 7. Index of International Reserves

In [175]:
# Index of International Reserves (value of reserves for January 1960 = 100)

monthly_germany.loc[:, 'International_Reserves_Index'] = (monthly_germany['International Reserves and Liquidity, Reserves, Official Reserve Assets, US Dollar'] / monthly_germany['International Reserves and Liquidity, Reserves, Official Reserve Assets, US Dollar'].iloc[1]) * 100

monthly_germany[['Month-Year', 'International_Reserves_Index']]

,Month-Year,International_Reserves_Index
0,1959-12-01,101.848342
1,1960-01-01,100.0
2,1960-02-01,101.740143
3,1960-03-01,105.128964
4,1960-04-01,110.837171
...,...,...
368,1990-08-01,1533.093782
369,1990-09-01,1549.431834
370,1990-10-01,1587.837706
371,1990-11-01,1612.268481


## Defining Variables (United States)
### 1. Monthly Inflation Rate


In [173]:
# Monthly inflation rate

monthly_us.loc[:, 'Monthly_Inflation_Rate (%)'] = monthly_us['Prices, Consumer Price Index, All items, Index'].pct_change() * 100

monthly_us[['Month-Year', 'Monthly_Inflation_Rate (%)']]

,Month-Year,Monthly_Inflation_Rate (%)
373,1959-12-01,NaN
374,1960-01-01,-0.340136
375,1960-02-01,0.341297
376,1960-03-01,0.0
377,1960-04-01,0.340136
...,...,...
741,1990-08-01,0.920245
742,1990-09-01,0.835866
743,1990-10-01,0.602864
744,1990-11-01,0.224719


### 2. Index of International Reserves

In [177]:
# Index of International Reserves (value of reserves for January 1960 = 100)

monthly_us.loc[:, 'International_Reserves_Index'] = (monthly_us['International Reserves and Liquidity, Reserves, Official Reserve Assets, US Dollar'] / monthly_us['International Reserves and Liquidity, Reserves, Official Reserve Assets, US Dollar'].iloc[1]) * 100

monthly_us[['Month-Year', 'International_Reserves_Index']]

,Month-Year,International_Reserves_Index
373,1959-12-01,100.122916
374,1960-01-01,100.0
375,1960-02-01,99.616353
376,1960-03-01,99.378902
377,1960-04-01,99.068353
...,...,...
741,1990-08-01,367.39208
742,1990-09-01,372.584941
743,1990-10-01,385.751982
744,1990-11-01,386.716715
